In [1]:
import pandas as pd
import numpy as np
import yaml
import sys, os, time
from os.path import join, isfile

file_name = 'trained_combined_model.yml'

In [2]:
def extract_models_params(data_path_list, file_name):
    row_list = []
    for data_path in data_path_list:
        grid_search_dict = yaml.load(open(join(data_path, file_name), 'r'))
        for elem in grid_search_dict.values():
            #print 'before appending:', row_list
            tmp_dict = elem['model_params']
            tmp_dict['model_threshold'] = elem['model_threshold']
            tmp_dict['model_type'] = elem['model_type']
            tmp_dict['result'] = elem['result']
            row_list.append(tmp_dict.copy())
            
    fea_imp_df = pd.DataFrame(row_list)
    print fea_imp_df.shape
    return fea_imp_df


In [54]:
#model_path = '/mnt/home/ymm/kaggle/bosch/full_data_FE_cross_fit_xgb_models/cross_validate_processed_subsample_250k_data_xgb_models'
#model_path = '/home/ymm/kaggle/bosch/full_data_FE_cross_fit_xgb_models/cross_validate_processed_subsample_100k_data_xgb_models'
#model_path = '/home/ymm/kaggle/bosch/full_data_FE_cross_fit_xgb_models/cross_validate_processed_subsample_50k_data_xgb_models_4'
#model_path = '/home/ymm/kaggle/bosch/full_data_FE_cross_fit_xgb_models/cross_validate_processed_subsample_50k_data_xgb_models_2'
model_path = '/home/ymm/kaggle/bosch/full_data_FE_cross_fit_xgb_models/cross_validate_processed_subsample_50k_data_xgb_models'

model_path = '/home/ymm/kaggle/bosch/full_data_FE_cross_fit_xgb_models/final_xgb_models/final_combined_xgb_models_1'
data_path_list = [model_path]

In [55]:
xgb_model_performance = extract_models_params(data_path_list, file_name)

(12, 15)


In [62]:
xgb_model_performance[xgb_model_performance['num_round'] == 701]

,colsample_bytree,eta,eval_metric,gamma,max_depth,metrics,model_threshold,model_type,num_round,result,seed,subsample,use_base_score,val,verbose_eval
0,0.5,0.0075,auc,0,5,auc,0.280361,Xgboost,701,0.408063,999,0.8,True,False,50
1,0.5,0.0075,auc,0,5,auc,0.353279,Xgboost,701,0.415516,999,0.8,True,False,50
10,0.5,0.0075,auc,0,5,auc,0.291691,Xgboost,701,0.409529,999,0.8,True,False,50
11,0.5,0.0075,auc,0,5,auc,0.248529,Xgboost,701,0.420956,999,0.8,True,False,50


In [57]:
performance_size = xgb_model_performance.groupby(['max_depth', 'num_round'])['result'].size()
#performance_size = xgb_model_performance.groupby(['max_depth', 'colsample_bytree'])['result'].size()
#performance_size = xgb_model_performance.groupby(['max_depth', 'num_round', 'colsample_bytree'])['result'].size()
performance_size

max_depth  num_round
3          501          4
4          801          4
5          701          4
dtype: int64

In [117]:
#xgb_model_performance.groupby(['num_round', 'max_depth'])['result'].mean()

In [58]:
grouped_results = pd.DataFrame()
grouped_results.loc[:, 'thres_mean'] = xgb_model_performance.groupby(['num_round', 'max_depth', 'colsample_bytree'])['model_threshold'].mean()
grouped_results.loc[:, 'thres_var'] = xgb_model_performance.groupby(['num_round', 'max_depth', 'colsample_bytree'])['model_threshold'].var()

grouped_results.loc[:, 'score_mean'] = xgb_model_performance.groupby(['num_round', 'max_depth', 'colsample_bytree'])['result'].mean()
grouped_results.loc[:, 'score_var'] = xgb_model_performance.groupby(['num_round', 'max_depth', 'colsample_bytree'])['result'].var()

In [59]:
grouped_results

,,,thres_mean,thres_var,score_mean,score_var
num_round,max_depth,colsample_bytree,,,,
501,3,0.7,0.354507,0.000810,0.396662,0.000148
701,5,0.5,0.293465,0.001924,0.413516,0.000035
801,4,0.5,0.271628,0.001962,0.409118,0.000402


In [61]:
grouped_results = grouped_results.sort_values(by='score_mean')
grouped_results

,,,thres_mean,thres_var,score_mean,score_var
num_round,max_depth,colsample_bytree,,,,
501,3,0.7,0.354507,0.000810,0.396662,0.000148
801,4,0.5,0.271628,0.001962,0.409118,0.000402
701,5,0.5,0.293465,0.001924,0.413516,0.000035


In [61]:
grouped_results = pd.DataFrame()
grouped_results.loc[:, 'mean'] = xgb_model_performance.groupby(['max_depth', 'colsample_bytree', 'num_round'])['result'].mean()
grouped_results.loc[:, 'var'] = xgb_model_performance.groupby(['max_depth', 'colsample_bytree', 'num_round'])['result'].var()

#xgb_model_performance.groupby(['max_depth', 'colsample_bytree', 'subsample'])['result'].mean()
#sorted_results = xgb_model_performance.groupby(['max_depth', 'subsample'])['result'].mean()
#xgb_model_performance.groupby(['max_depth', 'colsample_bytree'])['result'].mean()
#xgb_model_performance.groupby(['max_depth', 'colsample_bytree'])['result'].mean()
#sorted_results = xgb_model_performance.groupby(['max_depth'])['result'].mean()
#xgb_model_performance.groupby(['colsample_bytree'])['result'].mean()

In [62]:
grouped_results

mean       var
max_depth colsample_bytree num_round                    
3         0.5              401        0.367860  0.004221
                           501        0.363908  0.000926
                           601        0.376249  0.001621
          0.6              401        0.367023  0.000659
                           501        0.370303  0.002735
                           601        0.359612  0.000239
          0.7              401        0.372669  0.001941
                           501        0.379012  0.001328
                           601        0.371547  0.000105
4         0.5              401        0.357832  0.004134
                           501        0.379702  0.002576
                           601        0.359493  0.002279
          0.6              401        0.363247  0.001207
                           501        0.366150  0.000622
                           601        0.355796  0.002954
          0.7              401        0.377634  0.000496
                           501        0.372585  0.000419
                           601        0.368151  0.003006
5         0.5              401        0.385080  0.002253
                           501        0.367230  0.001533
                           601        0.362175  0.003605
          0.6              401        0.361244  0.001697
                           501        0.369228  0.001554
                           601        0.348474  0.001252
          0.7              401        0.378074  0.002704
                           501        0.371491  0.002220
                           601        0.371377  0.000965
6         0.5              401        0.379096  0.002512
                           501        0.361178  0.001071
                           601        0.370929  0.002080
          0.6              401        0.375016  0.001744
                           501        0.374778  0.000635
                           601        0.354402  0.001129
          0.7              401        0.375790  0.005340
                           501        0.368483  0.000578
                           601        0.378925  0.004387

In [63]:
grouped_results = grouped_results.sort_values(by='mean')

In [65]:
grouped_results

mean       var
max_depth colsample_bytree num_round                    
5         0.6              601        0.348474  0.001252
6         0.6              601        0.354402  0.001129
4         0.6              601        0.355796  0.002954
          0.5              401        0.357832  0.004134
                           601        0.359493  0.002279
3         0.6              601        0.359612  0.000239
6         0.5              501        0.361178  0.001071
5         0.6              401        0.361244  0.001697
          0.5              601        0.362175  0.003605
4         0.6              401        0.363247  0.001207
3         0.5              501        0.363908  0.000926
4         0.6              501        0.366150  0.000622
3         0.6              401        0.367023  0.000659
5         0.5              501        0.367230  0.001533
3         0.5              401        0.367860  0.004221
4         0.7              601        0.368151  0.003006
6         0.7              501        0.368483  0.000578
5         0.6              501        0.369228  0.001554
3         0.6              501        0.370303  0.002735
6         0.5              601        0.370929  0.002080
5         0.7              601        0.371377  0.000965
                           501        0.371491  0.002220
3         0.7              601        0.371547  0.000105
4         0.7              501        0.372585  0.000419
3         0.7              401        0.372669  0.001941
6         0.6              501        0.374778  0.000635
                           401        0.375016  0.001744
          0.7              401        0.375790  0.005340
3         0.5              601        0.376249  0.001621
4         0.7              401        0.377634  0.000496
5         0.7              401        0.378074  0.002704
6         0.7              601        0.378925  0.004387
3         0.7              501        0.379012  0.001328
6         0.5              401        0.379096  0.002512
4         0.5              501        0.379702  0.002576
5         0.5              401        0.385080  0.002253

In [59]:
grouped_results_var = xgb_model_performance.groupby(['max_depth', 'colsample_bytree', 'num_round'])['result'].var()
grouped_results_var = grouped_results_var.sort_values()
grouped_results_var

max_depth  colsample_bytree  num_round
3          0.7               601          0.000105
           0.6               601          0.000239
4          0.7               501          0.000419
                             401          0.000496
6          0.7               501          0.000578
4          0.6               501          0.000622
6          0.6               501          0.000635
3          0.6               401          0.000659
           0.5               501          0.000926
5          0.7               601          0.000965
6          0.5               501          0.001071
           0.6               601          0.001129
4          0.6               401          0.001207
5          0.6               601          0.001252
3          0.7               501          0.001328
5          0.5               501          0.001533
           0.6               501          0.001554
3          0.5               601          0.001621
5          0.6               401          0

In [ ]:
sorted_results = xgb_model_performance.groupby(['max_depth', 'colsample_bytree', 'num_round'])['result'].mean()
#sorted_results = xgb_model_performance.groupby(['max_depth', 'colsample_bytree'])['result'].mean()
sorted_results.sort()
sorted_results

### RandomForest Models 

In [ ]:
'''
general_path = '/home/ymm/kaggle/bosch/full_data_FE_filled_missing_models/RandomForest_models'

subFolder_pathList = ['cross_validate_processed_subsample_50k_data_rf_models_0',
                      'cross_validate_processed_subsample_50k_data_rf_models_1',
                      'cross_validate_processed_subsample_50k_data_rf_models_2']

pathList = [join(general_path, path) for path in subFolder_pathList]
file_name = 'trained_combined_model.yml'

rf_model_performance = extract_models_params(pathList, file_name)
'''

In [42]:
#rf_model_path = '/home/ymm/kaggle/bosch/full_data_FE_filled_missing_models/RandomForest_models/cross_validate_processed_subsample_50k_data_rf_models_0'
#rf_model_path = '/home/ymm/kaggle/bosch/full_data_FE_filled_missing_models/RandomForest_models/cross_validate_processed_subsample_50k_data_rf_models_1'
rf_model_path = '/home/ymm/kaggle/bosch/full_data_FE_filled_missing_models/RandomForest_models/cross_validate_processed_subsample_50k_data_rf_models_2'
#rf_model_path = '/home/ymm/kaggle/bosch/full_data_FE_filled_missing_models/RandomForest_models/cross_validate_processed_subsample_50k_data_rf_models_3'
#rf_model_path = '/home/ymm/kaggle/bosch/full_data_FE_filled_missing_models/RandomForest_models/cross_validate_processed_subsample_50k_data_rf_models'
data_path_list = [rf_model_path]
rf_model_performance = extract_models_params(data_path_list, file_name)

(50, 19)


In [43]:
rf_model_performance = rf_model_performance.sort_values(by='result')

In [44]:
print rf_model_performance['model_type'].value_counts()
rf_model_performance[rf_model_performance['model_type'] == 'Xgboost']

RandomForest    48
Xgboost          2
Name: model_type, dtype: int64


,colsample_bytree,criterion,eta,eval_metric,gamma,max_depth,metrics,model_threshold,model_type,n_estimators,n_jobs,num_round,random_state,result,seed,subsample,use_base_score,val,verbose_eval
20,0.8,NaN,0.0075,auc,0.0,5,auc,0.250171,Xgboost,NaN,NaN,501.0,NaN,0.446138,999.0,0.8,True,False,50.0
19,0.8,NaN,0.0075,auc,0.0,5,auc,0.153743,Xgboost,NaN,NaN,501.0,NaN,0.461889,999.0,0.8,True,False,50.0


In [45]:
rf_model_performance.head(20)

,colsample_bytree,criterion,eta,eval_metric,gamma,max_depth,metrics,model_threshold,model_type,n_estimators,n_jobs,num_round,random_state,result,seed,subsample,use_base_score,val,verbose_eval
47,NaN,gini,NaN,NaN,NaN,20,NaN,0.172500,RandomForest,2000.0,-1.0,NaN,9999.0,0.146918,NaN,NaN,NaN,NaN,NaN
1,NaN,gini,NaN,NaN,NaN,7,NaN,0.077162,RandomForest,2000.0,-1.0,NaN,9999.0,0.168294,NaN,NaN,NaN,NaN,NaN
18,NaN,gini,NaN,NaN,NaN,7,NaN,0.078553,RandomForest,3000.0,-1.0,NaN,9999.0,0.217340,NaN,NaN,NaN,NaN,NaN
22,NaN,gini,NaN,NaN,NaN,5,NaN,0.035268,RandomForest,500.0,-1.0,NaN,9999.0,0.234384,NaN,NaN,NaN,NaN,NaN
42,NaN,gini,NaN,NaN,NaN,10,NaN,0.076037,RandomForest,2000.0,-1.0,NaN,9999.0,0.247207,NaN,NaN,NaN,NaN,NaN
31,NaN,gini,NaN,NaN,NaN,5,NaN,0.125930,RandomForest,1000.0,-1.0,NaN,9999.0,0.256075,NaN,NaN,NaN,NaN,NaN
11,NaN,gini,NaN,NaN,NaN,20,NaN,0.125000,RandomForest,3000.0,-1.0,NaN,9999.0,0.257475,NaN,NaN,NaN,NaN,NaN
15,NaN,gini,NaN,NaN,NaN,10,NaN,0.127959,RandomForest,3000.0,-1.0,NaN,9999.0,0.270726,NaN,NaN,NaN,NaN,NaN
45,NaN,gini,NaN,NaN,NaN,25,NaN,0.114000,RandomForest,2000.0,-1.0,NaN,9999.0,0.270726,NaN,NaN,NaN,NaN,NaN
13,NaN,gini,NaN,NaN,NaN,15,NaN,0.191333,RandomForest,3000.0,-1.0,NaN,9999.0,0.270726,NaN,NaN,NaN,NaN,NaN


In [46]:
rf_model_performance = rf_model_performance.loc[rf_model_performance['model_type'] == 'RandomForest', :]

In [47]:
performance_size = rf_model_performance.groupby(['max_depth', 'n_estimators'])['result'].size()

In [48]:
performance_size

max_depth  n_estimators
5          500.0           2
           1000.0          2
           2000.0          2
           3000.0          2
7          500.0           2
           1000.0          2
           2000.0          2
           3000.0          2
10         500.0           2
           1000.0          2
           2000.0          2
           3000.0          2
15         500.0           2
           1000.0          2
           2000.0          2
           3000.0          2
20         500.0           2
           1000.0          2
           2000.0          2
           3000.0          2
25         500.0           2
           1000.0          2
           2000.0          2
           3000.0          2
dtype: int64

In [49]:
rf_model_performance.groupby(['max_depth', 'n_estimators'])['result'].mean()

max_depth  n_estimators
5          500.0           0.305720
           1000.0          0.349529
           2000.0          0.272646
           3000.0          0.346590
7          500.0           0.380148
           1000.0          0.385549
           2000.0          0.356175
           3000.0          0.301504
10         500.0           0.370156
           1000.0          0.361849
           2000.0          0.308257
           3000.0          0.335031
15         500.0           0.326552
           1000.0          0.401006
           2000.0          0.380742
           3000.0          0.276208
20         500.0           0.297357
           1000.0          0.362942
           2000.0          0.304403
           3000.0          0.278911
25         500.0           0.393728
           1000.0          0.371186
           2000.0          0.366308
           3000.0          0.352706
Name: result, dtype: float64

In [50]:
sorted_performance = pd.DataFrame()
sorted_performance['mean'] = rf_model_performance.groupby(['max_depth', 'n_estimators'])['result'].mean()
sorted_performance['var'] = rf_model_performance.groupby(['max_depth', 'n_estimators'])['result'].var()
sorted_performance

mean       var
max_depth n_estimators                    
5         500.0         0.305720  0.010177
          1000.0        0.349529  0.017467
          2000.0        0.272646  0.000001
          3000.0        0.346590  0.003125
7         500.0         0.380148  0.000736
          1000.0        0.385549  0.007342
          2000.0        0.356175  0.070598
          3000.0        0.301504  0.014167
10        500.0         0.370156  0.002481
          1000.0        0.361849  0.014209
          2000.0        0.308257  0.007454
          3000.0        0.335031  0.008270
15        500.0         0.326552  0.001803
          1000.0        0.401006  0.026108
          2000.0        0.380742  0.000547
          3000.0        0.276208  0.000060
20        500.0         0.297357  0.000018
          1000.0        0.362942  0.000081
          2000.0        0.304403  0.049603
          3000.0        0.278911  0.000919
25        500.0         0.393728  0.000130
          1000.0        0.371186  0.008019
          2000.0        0.366308  0.018272
          3000.0        0.352706  0.000264

In [51]:
sorted_performance = sorted_performance.sort_values(by='mean')
sorted_performance

mean       var
max_depth n_estimators                    
5         2000.0        0.272646  0.000001
15        3000.0        0.276208  0.000060
20        3000.0        0.278911  0.000919
          500.0         0.297357  0.000018
7         3000.0        0.301504  0.014167
20        2000.0        0.304403  0.049603
5         500.0         0.305720  0.010177
10        2000.0        0.308257  0.007454
15        500.0         0.326552  0.001803
10        3000.0        0.335031  0.008270
5         3000.0        0.346590  0.003125
          1000.0        0.349529  0.017467
25        3000.0        0.352706  0.000264
7         2000.0        0.356175  0.070598
10        1000.0        0.361849  0.014209
20        1000.0        0.362942  0.000081
25        2000.0        0.366308  0.018272
10        500.0         0.370156  0.002481
25        1000.0        0.371186  0.008019
7         500.0         0.380148  0.000736
15        2000.0        0.380742  0.000547
7         1000.0        0.385549  0.007342
25        500.0         0.393728  0.000130
15        1000.0        0.401006  0.026108